In [1]:
import pandas as pd
import json
import pyarrow
import ast

In [2]:
# Empezamos pasando el archivo json a un dataframe
dataList = []
filePath = 'DB Steam/user_reviews.json'

# Abriendo el archivo y procesaa cada línea
with open(filePath, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            jsonData = ast.literal_eval(line)
            dataList.append(jsonData)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Se crea el DataFrame a partir de la lista de diccionarios
dfReviews = pd.DataFrame(dataList)

In [3]:
dfReviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [4]:
# Se eliminan las filas que esten vacias para después convertir el dataframe a parquet
dfReviews = dfReviews.dropna(how='all')
dfReviews.to_parquet('DB Steam/Reviews.parquet', index = False)

In [5]:
# Se usa el metodo explode para desanidar los listas de diccionarios de la columna reviews y crear nuevas filas con la información de cada review
dfReviews_sep = dfReviews.explode('reviews').reset_index(drop = True)

In [6]:
dfReviews_sep

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [7]:
# Para observar lo que contiene cada fila de la columna review, traemos una de ellas
dfReviews_sep['reviews'][2]

{'funny': '',
 'posted': 'Posted April 21, 2011.',
 'last_edited': '',
 'item_id': '43110',
 'helpful': 'No ratings yet',
 'recommend': True,
 'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}

In [8]:
# Se crea una nueva columna de nombre 'review' y se extrae el texto de la reseña de la columna 'reviews' 
# Para estas extracciones usaremos lambda como una forma directa y rápida de crear la función
dfReviews_sep['review'] = dfReviews_sep['reviews'].apply(lambda x: x.get('review') if isinstance(x, dict) and 'review' in x else None)

# Se eliminan las filas vacías
dfReviews_sep = dfReviews_sep.dropna(subset='review')

# Se crea una nueva columna y se extrae la recomendación, este es un valor booleano
dfReviews_sep['recommend'] = dfReviews_sep['reviews'].apply(lambda x: x.get('recommend') if isinstance(x, dict) and 'recommend' in x else None)

# Se crea una nueva columna y se extrae el 'item_id' que nos servirá para vincular este dataframe con el resto
dfReviews_sep['item_id'] = dfReviews_sep['reviews'].apply(lambda x: x.get('item_id') if isinstance(x, dict) and 'item_id' in x else None)

# Se crea una nueva columna y se extrae el la fecha de la reseña que nos servirá para vincular este dataframe con el resto
dfReviews_sep['review_date'] = dfReviews_sep['reviews'].apply(lambda x: x.get('posted') if isinstance(x, dict) and 'posted' in x else None)

# Se elimina la parte de la fecha que contiene el string 'Posted ' para que pueda transformarse a formato de fecha
# Para esto se reemplaza el string 'Posted' por ''
dfReviews_sep['review_date'] = dfReviews_sep['review_date'].str.replace('Posted ', '', regex=False)

# Una vez tenemos solo los datos de la fecha, convertimos en formato datetime
dfReviews_sep['review_date'] = pd.to_datetime(dfReviews_sep['review_date'],format='mixed',errors='coerce')

# De la columna de 'review_date' se extrae el dato del año y se transforma a formato numérico
dfReviews_sep['review_year'] = dfReviews_sep['review_date'].dt.year
dfReviews_sep['review_year'] = pd.to_numeric(dfReviews_sep['review_year'], errors='coerce').astype('Int64')

/var/folders/47/k8p8b34120x38zvvxtyv0l6c0000gn/T/ipykernel_1444/1573015643.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfReviews_sep['recommend'] = dfReviews_sep['reviews'].apply(lambda x: x.get('recommend') if isinstance(x, dict) and 'recommend' in x else None)
/var/folders/47/k8p8b34120x38zvvxtyv0l6c0000gn/T/ipykernel_1444/1573015643.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfReviews_sep['item_id'] = dfReviews_sep['reviews'].apply(lambda x: x.get('item_id') if isinstance(x, dict) and '

In [9]:
# Verificamos como ha quedado el dataframe 
dfReviews_sep

,user_id,user_url,reviews,review,recommend,item_id,review_date,review_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",Simple yet with great replayability. In my opi...,True,1250,2011-11-05,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",It's unique and worth a playthrough.,True,22200,2011-07-15,2011
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",Great atmosphere. The gunplay can be a bit chu...,True,43110,2011-04-21,2011
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",I know what you think when you see this title ...,True,251610,2014-06-24,2014
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",For a simple (it's actually not all that simpl...,True,227300,2013-09-08,2013
...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la...",a must have classic from steam definitely wort...,True,70,NaT,<NA>
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las...",this game is a perfect remake of the original ...,True,362890,NaT,<NA>
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ...",had so much fun plaing this and collecting res...,True,273110,NaT,<NA>
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la...",:D,True,730,NaT,<NA>


In [10]:
# Seguidamente transformamos el dataframe a formato parquet 
dfReviews_sep.to_parquet('DB Steam/Reviews_sep.parquet', index = False)